# **Train a detector for MOT**

# RetinaNet

In [1]:
import mmcv
from mmdet.apis import set_random_seed
cfg = mmcv.Config.fromfile('./configs/det/retinanet_r50_fpn_visdrone.py')

cfg.work_dir = './tutorial_exps/retina_bytetrack_16b_2e_notrunc_mixup_mosaic/' # 모델 이름에 맞게 변경 요망
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'
cfg.total_epochs = 2
cfg.data.samples_per_gpu = 16
cfg.workers_per_gpu = 4
print(f'Config:\n{cfg.pretty_text}')

/home/ljj3499/anaconda3/envs/mmtracking/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


Config:
model = dict(
    detector=dict(
        type='RetinaNet',
        backbone=dict(
            type='ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=True,
            style='pytorch',
            init_cfg=dict(
                type='Pretrained', checkpoint='torchvision://resnet50')),
        neck=dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            start_level=1,
            add_extra_convs='on_input',
            num_outs=5),
        bbox_head=dict(
            type='RetinaHead',
            num_classes=5,
            in_channels=256,
            stacked_convs=4,
            feat_channels=256,
            anchor_generator=dict(
                type='AnchorGenerator',
                octave_base_scale=4,
                scales_per_octave=3,
                rat

In [2]:
import os.path as osp
from mmtrack.datasets import build_dataset
from mmdet.apis import train_detector as train_model
from mmdet.models import build_detector as build_model

mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
model = build_model(cfg.model.detector)
model.init_weights()
datasets = [build_dataset(cfg.data.train)]

model.CLASSES = datasets[0].CLASSES
train_model(model, datasets, cfg)

2023-08-08 10:23:21,870 - mmcv - INFO - initialize RetinaNet with init_cfg {'type': 'Pretrained', 'checkpoint': '~/sia/mmtracking/checkpoints/retinanet_r50_fpn_fp16_1x_coco_20200702-0dbfb212.pth'}
2023-08-08 10:23:21,871 - mmcv - INFO - load model from: ~/sia/mmtracking/checkpoints/retinanet_r50_fpn_fp16_1x_coco_20200702-0dbfb212.pth
2023-08-08 10:23:21,873 - mmcv - INFO - load checkpoint from local path: ~/sia/mmtracking/checkpoints/retinanet_r50_fpn_fp16_1x_coco_20200702-0dbfb212.pth
2023-08-08 10:23:21,950 - mmcv - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.retina_cls.weight: copying a param with shape torch.Size([720, 256, 3, 3]) from checkpoint, the shape in current model is torch.Size([45, 256, 3, 3]).
size mismatch for bbox_head.retina_cls.bias: copying a param with shape torch.Size([720]) from checkpoint, the shape in current model is torch.Size([45]).


loading annotations into memory...
Done (t=5.01s)
creating index...
index created!


/home/ljj3499/anaconda3/envs/mmtracking/lib/python3.9/site-packages/mmdet-2.28.2-py3.9.egg/mmdet/utils/compat_config.py:28: UserWarning: config is now expected to have a `runner` section, please set `runner` in your config.
  warnings.warn(
2023-08-08 10:23:30,036 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2023-08-08 10:23:30,038 - mmdet - INFO - Start running, host: ljj3499@aiffel-online04-demasia, work_dir: /home/ljj3499/sia/mmtracking/tutorial_exps/retina_bytetrack_16b_2e_notrunc_mixup_mosaic
2023-08-08 10:23:30,039 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 ----------------